In [1]:
import torch
import torch.nn as nn
import time
from tqdm import tqdm
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights
import sys
sys.path.append(sys.path[0]+'/../')
sys.path.append(sys.path[0]+'/../utils')
from models.denseFuse import DenseFuse
from models.vae import Variational_Autoencoder
from dataloaders.minc_dataloader import MINCDataset
from dataloaders.data_loader import VAEDataset, VAEDataset1
from utils import Combined_Loss, Combined_Loss2
from loss_resnet import LossRN
from models.opticFlow import OF_AE, pyramid

In [2]:
BATCH_SIZE_TRAIN = 1
BATCH_SIZE_TEST = 1
LOAD = False
train_save = open("../logs/train_loss_of.txt", "a")
test_save = open("../logs/test_loss_of.txt", "a")

#model = OF_AE().cuda()
model = Variational_Autoencoder().cuda()
#t = vae.train()
if LOAD:
    model.load_state_dict(torch.load("of.pth"))
    t = model.eval()
else:
    t = model.train()
#rgb_p = 'archive/FLIR_ADAS_1_3/train/RGB/'
#ir__p = 'archive/FLIR_ADAS_1_3/train/thermal_8_bit/'

rgb_p = '../datasets/irh/img_raw/rgb/'
ir__p = '../datasets/irh/img_raw/nir/'

train_ds = VAEDataset1(rgb_p, ir__p)
train_loader = DataLoader(
        dataset=train_ds,
        batch_size=BATCH_SIZE_TRAIN,
        num_workers=0,
        pin_memory=True,
        shuffle=True
    )

#rgb_vp = 'archive/FLIR_ADAS_1_3/val/RGB/'
#ir__vp = 'archive/FLIR_ADAS_1_3/val/thermal_8_bit/'
rgb_vp = '../datasets/irh/img_raw/rgb/'
ir__vp = '../datasets/irh/img_raw/nir/'

val_ds = VAEDataset1(rgb_vp, ir__vp)
val_loader = DataLoader(
        dataset=val_ds,
        batch_size=BATCH_SIZE_TEST,
        num_workers=0,
        pin_memory=True,
        shuffle=True
    )


optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss = Combined_Loss2()
#summary(vae, [(1,256,256), (3,256,256)])

In [3]:
for epc in range(0,200):
    epoch_loss = []
    r = tqdm(enumerate(train_loader), leave=False, desc="Epoch: {}".format(epc), total=len(train_loader))
    for idx, (ir, rgb) in r:
        ir, rgb = ir.cuda(), rgb.cuda()
        If = model(ir, rgb)
        lf = loss.compute(ir, rgb, If[0])
        epoch_loss.append(lf.item())
        del rgb, ir, If
        torch.cuda.empty_cache()
        optimizer.zero_grad()
        lf.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type=1)
        optimizer.step()
        r.set_postfix(loss=lf.item())
    #save epoch loss list
    train_save.write("epoch [{}] = ".format(epc) + str(epoch_loss) + '\n')
    #save checkpoint
    torch.save(model.state_dict(), "of.pth")
    '''
    #test loss run on val
    with torch.no_grad():
        e = mode.eval()
        ir, rgb = next(iter(val_loader))
        ir, rgb = pyramid(ir, rgb)
        If = model(rgb, ir)
        lf = loss2.compute(ir[0], rgb[0], If)
        print("epoch [{}] = ".format(epc) + str(float(lf)))
        test_save.write("epoch [{}] = ".format(epc) + str(float(lf)) + '\n')
        
        del rgb, ir, If
        torch.cuda.empty_cache()
    ''' 

KeyboardInterrupt: 